In [1]:
from gsheets import Sheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import math
import datetime
from datetime import timedelta  
import emoji
import re
import warnings
import statistics
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import infrastructure 
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import filepath
import smartsheet
import os 


# Downloading Lexi, Offers, CW, CT, and Mamba

In [2]:
lexi_original = infrastructure.get_lexi()
lexi = lexi_original.copy()
lexi = lexi[lexi['Delivered']>0]
lexi = infrastructure.transform_sms_df(lexi)

lexi['eCPM'] = lexi['Revenue']*1000 / lexi['Delivered']
lexi['CTR'] = lexi['Clicks'] / lexi['Delivered']
lexi['Shortcode Name'] = lexi['Shortcode Name'].str[:3]

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_13320/4212767309.py:1: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  lexi_original = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [3]:
cw = infrastructure.get_lanina()
cw.dropna(subset=['Reporting Content ID'],inplace=True)

cw.dropna(subset=['OfferIDs'], inplace = True)
mask = cw['Type (Pitch)'] == 'HOL'
cw = cw[~mask]
cw['Allocation Period (Date Added)'] = pd.to_datetime(cw['Allocation Period (Date Added)'], format='mixed')
cw.dropna(subset=['Reporting Content ID'],inplace=True)
cw['OfferIDs'] = cw['OfferIDs'].astype(str)

In [4]:
offers_original = infrastructure.get_smartsheet('offers_sms')
offers = offers_original.copy()
offers.dropna(subset=['Hitpath Offer ID'],inplace=True)
offers['Hitpath Offer ID'] = offers['Hitpath Offer ID'].astype(int)
offers = offers.loc[(offers['Hitpath Offer ID']>1000) & (offers['Hitpath Offer ID'].isna()==False)]
offers['Hitpath Offer ID'] = offers['Hitpath Offer ID'].astype(str)

In [5]:
ow = infrastructure.get_smartsheet('ow_sms')
ow['Available Shortcode'] = ow['Available Shortcode'].apply(lambda x: x.split('\n'))
ow_status_dict = ow.set_index('Hitpath Offer ID')['Status'].to_dict()
live_ow_shortcodes_dict = ow[ow['Status']=='Live'].set_index('Hitpath Offer ID')['Available Shortcode'].to_dict()
not_live_ows = ow[~(ow['Status']=='Live')]['Hitpath Offer ID'].unique().tolist()
ow_hitpath_list = ow['Hitpath Offer ID'].unique().tolist()

In [6]:
lexi = lexi.sort_values(by='Date')

lexi_30 = lexi[(lexi['Date'].dt.date >= (date.today() + timedelta(days=-30)))].copy()
lexi_90 = lexi[(lexi['Date'].dt.date >= (date.today() + timedelta(days=-90)))].copy()

In [7]:
ct =infrastructure.get_smartsheet('content_sms')
ct['Submission Date'] = pd.to_datetime(ct['Submission Date'])
ct['Date Added to La Nina (Akshad)'].replace(to_replace=r"[a-zA-Z- ()]", value="", regex=True, inplace=True)
ct['Date Added to La Nina (Akshad)'] = pd.to_datetime(ct['Date Added to La Nina (Akshad)'], errors='coerce')

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_13320/3074324814.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ct['Submission Date'] = pd.to_datetime(ct['Submission Date'])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_13320/3074324814.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ct['Date Added to La Nina (Akshad)'] = pd.to_datetime(ct['Date Added to La Nina (Akshad)'], errors='coerce')


In [8]:
mamba = infrastructure.get_mamba()
mamba['Affiliate ID']=mamba['Dataset'].str.split('_').str[2]
mamba[['Limit','Offset']] = mamba[['Limit','Offset']].fillna("")
mamba['Drop Number'] = mamba['Drop'].str.split(expand=True)[1].astype(int)
mamba['Day of Week'] = mamba['Date'].dt.dayofweek
mamba.rename(columns={"Segment ": "Segment"},inplace=True)

/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_13320/2744609448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mamba['Affiliate ID']=mamba['Dataset'].str.split('_').str[2]
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_13320/2744609448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [9]:
eligible_hitpaths = offers[(offers['Status'].isin(['Live']))&(offers['Direct Traffic Accepted ']=='Yes')]['Hitpath Offer ID'].unique().tolist()

In [10]:
tf_codes = ['MFA', 'N3G', "FRH","TCH","THM","UTL"]

In [11]:
tf_cw = cw[cw['Type'].isin(tf_codes)].copy()

# Functions

In [12]:
def find_already_requested_tf_content():
    """
    :return: a list of Hitpath Offer IDs that have had TF content requested within the past 3 weeks in the 
            SMS Creative Submission smartsheet.
    """
    already_requested_content = ct.copy()
    already_requested_content = already_requested_content[['Submission Date', 'Date Added to La Nina (Akshad)', '# of Pieces Requested Per Toll Free', 'Hitpath Offer ID', 'Request Type (Request)','Toll Free Abbreviations']]
    already_requested_content = already_requested_content[already_requested_content['Request Type (Request)'].str.contains('Content', na=False) ]
    # Any offer without a date in this column has not been added to La Nina, and should not be requested again
    already_requested_content = already_requested_content[((pd.isna(already_requested_content['Date Added to La Nina (Akshad)']))&(already_requested_content['Submission Date']>=pd.Timestamp(date.today()-timedelta(30)))) | 
                                                          (already_requested_content['Date Added to La Nina (Akshad)']>=pd.Timestamp(date.today() - timedelta(21)))]
    already_requested_content = already_requested_content[ already_requested_content['# of Pieces Requested Per Toll Free']>0]                                                      

    return already_requested_content['Hitpath Offer ID'].unique().tolist()

In [13]:
def make_base_df():
    """ Make a dataframe containing all possible Hitpath/TF Shortcode combos"""

    # Combine eligible_hitpaths with Toll Free Shortcodes
    base_index = pd.MultiIndex.from_product([eligible_hitpaths, tf_codes], names=['Hitpath Offer ID', 'Shortcode Name'])

    base_df = pd.DataFrame(index=base_index)

    return base_df    

In [14]:
def get_CTR50s():
    """Calculates CTR50 for each Hitpath for all Lexi data"""
    
    CTR50s = lexi.groupby(['Hitpath Offer ID'])[['Clicks', 'Revenue', 'Delivered']].sum()
    CTR50s['eCPM'] = 1000 * CTR50s['Revenue'] / CTR50s['Delivered']
    CTR50s['CTR'] = CTR50s['Clicks'] / CTR50s['Delivered']
    CTR50s['eCPM Normalized'] = (CTR50s['eCPM'] - CTR50s['eCPM'].min()) / (CTR50s['eCPM'].max() - CTR50s['eCPM'].min())
    CTR50s['CTR Normalized'] = (CTR50s['CTR'] - CTR50s['CTR'].min()) / (CTR50s['CTR'].max() - CTR50s['CTR'].min())
    CTR50s['CTR50'] = CTR50s['eCPM Normalized'] + CTR50s['CTR Normalized']

    eCPM0_offers = []
    for index, row in CTR50s.iterrows():
        if row['eCPM'] == 0:
            eCPM0_offers.append(index)

    eCPM0_offers = pd.DataFrame(eCPM0_offers, columns=['Hitpath Offer ID'])
    eCPM0_offers['Reason for Exclusion'] = 'Aggregate eCPM is $0'
    
    return CTR50s, eCPM0_offers

In [15]:
def create_stats_by_dayrange(dayrange):
    """Creates statistics given the previous days you want to look at.
    
    :param dayrange: an int of the number of days in the past in Lexi you want to consider
    :return: a DataFrame of lexi stats for each Hitpath in the number of days given"""
    
    shortened_lexi = lexi[(lexi['Date'].dt.date >= (date.today() - timedelta(days=dayrange)))].copy()
    sum_stats = ['Revenue', 'Delivered', 'Optout', 'Clicks', 'Jump Page Clicks', 'Opportunity Cost']
    stats_df = shortened_lexi.groupby(['Hitpath Offer ID', 'Shortcode Name'])[sum_stats].sum()
    stats_df['eCPM'] = 1000 * stats_df['Revenue'] / stats_df['Delivered']
    stats_df['OOR'] = stats_df['Optout'] / stats_df['Delivered']
    stats_df['CTR'] = stats_df['Clicks'] / stats_df['Delivered']
    stats_df['Jump Page CTR'] = stats_df['Jump Page Clicks'] / stats_df['Delivered']
    stats_df['Opportunity Cost eCPM'] = 1000 * stats_df['Opportunity Cost'] / stats_df['Delivered']
    stats_df['Engagement Ratio'] = stats_df['CTR'] / stats_df['OOR']
    shortened_lexi = shortened_lexi[shortened_lexi['Creative Type'].isna() == False]
    extra_info = shortened_lexi.groupby(['Hitpath Offer ID', 'Shortcode Name']).agg(Drops=('Date', 'count'), Max_Date=('Date', 'max'), Creative=('Creative Type', lambda x: x.value_counts().idxmax() if len(x)>0 else "no Data"), 
                                                                                    Creative_Drops=('Creative Type', lambda x: x.value_counts().max()))
    extra_info = extra_info.rename(columns={'Drops':'Number of Drops', 'Max_Date':'Most Recent Drop', 'Creative':'Most Dropped Creative'})
    extra_info['Top Creative Drop %'] = round(extra_info['Creative_Drops'] / extra_info['Number of Drops'], 4)
    extra_info = extra_info.drop('Creative_Drops', axis=1)
    stats_df = stats_df.merge(extra_info, how='left', left_index=True, right_index=True)

    return stats_df

In [16]:
def make_final_dfs(min_pieces=2):
    """Makes the DataFrames necessary for the report.
    
    :param min_pieces: an int of the number of Pieces you want to have for each hitpath. Default is 2
    :return: 3 DataFrames. The first is a DataFrame of Hitpaths that need requests. The second is a DataFrame
    of 30 day statistics for each Hitpath/Shortcode combo. The third is a DataFrame of hitpaths that were excluded from being requested despite
    being eligible based on Offer smartsheet requirements """

    base_df = make_base_df()
    base_df = base_df.reset_index(level=1)
    CTR50s, eCPM0_offers = get_CTR50s()
    already_requested_hitpaths = find_already_requested_tf_content()

    eCPM0_offers = eCPM0_offers[eCPM0_offers['Hitpath Offer ID'].isin(eligible_hitpaths)]

    hitpaths_to_exclude = eCPM0_offers['Hitpath Offer ID'].tolist()

    # Merge CTR50 info onto base_df
    base_df = base_df.merge(CTR50s, how='left', on='Hitpath Offer ID')
    base_df = base_df.reset_index()
    base_df = base_df.set_index(['Hitpath Offer ID', 'Shortcode Name'])

    # Find how many content pieces there are currently for each offer
    num_current_pieces = tf_cw.groupby(['OfferIDs', 'Type']).agg(num_current_pieces=('Reporting Content ID', 'count'))
    num_current_pieces = num_current_pieces.rename(columns={'num_current_pieces':'Pieces in La Nina'})
    num_current_pieces.index.names = ['Hitpath Offer ID', 'Shortcode Name']

    # Show how many pieces currently in La Nina for each Hit/Shortcode combo
    base_df = base_df.merge(num_current_pieces, how='left', left_index=True, right_index=True)    
    # If N/A, not in La Nina, so value is 0
    base_df['Pieces in La Nina'] = base_df['Pieces in La Nina'].fillna(0)
    # Determine how many pieces to request
    base_df['Pieces to Request'] = min_pieces - base_df['Pieces in La Nina']

    # Get list of Hitpath Offer IDs that have any of their shortcodes needing more pieces
    hitpaths_to_request = base_df[base_df['Pieces in La Nina'] < min_pieces].groupby(level='Hitpath Offer ID').any().index.tolist()

    # Do not request any offers with $0 aggregate eCPM
    hitpaths_to_request = [hitpath for hitpath in hitpaths_to_request if hitpath not in eCPM0_offers]

    # Dataframe of only Hitpath Offer ID/Shortcode combos that need requests
    need_request = base_df.loc[hitpaths_to_request].copy()
    need_request['Pieces to Request'] = need_request.groupby(level='Hitpath Offer ID')['Pieces to Request'].transform('max')
    need_request = need_request.sort_values(by=['CTR50', 'Pieces to Request'], ascending=[False, False])
    need_request = need_request[['Pieces to Request', 'CTR50']]
    need_request = need_request.reset_index()
    need_request = need_request[~(need_request['Hitpath Offer ID'].isin(hitpaths_to_exclude))]
    need_request = need_request[~(need_request['Hitpath Offer ID'].isin(already_requested_hitpaths))]
    need_request = need_request.drop('Shortcode Name', axis=1)
    need_request = need_request.drop_duplicates(subset='Hitpath Offer ID')

    # Make df of stats
    stats_30d = create_stats_by_dayrange(30)
    # Merge 30d stats onto base df (just using Pieces in La Nina as placeholder)
    tf_30d_stats = base_df[['Pieces in La Nina']].merge(stats_30d, how='left', left_index=True, right_index=True)
    tf_30d_stats = tf_30d_stats[stats_30d.columns]
    tf_30d_stats = tf_30d_stats.reset_index()
    tf_30d_stats = tf_30d_stats.sort_values(by=['eCPM', 'Hitpath Offer ID'], ascending=[False, True])

    # Make DataFrame of hitpaths that were excluded from being requested
    already_requested_hitpaths_df = pd.DataFrame(already_requested_hitpaths, columns=['Hitpath Offer ID'])
    already_requested_hitpaths_df['Reason for Exclusion'] = 'Hitpath already has TF content requested'
    excluded_hitpath_requests = pd.concat([eCPM0_offers, already_requested_hitpaths_df])
    excluded_hitpath_requests = excluded_hitpath_requests.merge(base_df['Pieces in La Nina'], how='left', on='Hitpath Offer ID')
    excluded_hitpath_requests['Pieces in La Nina'] = excluded_hitpath_requests['Pieces in La Nina'].fillna(0)
    excluded_hitpath_requests = excluded_hitpath_requests.drop_duplicates(subset=['Hitpath Offer ID', 'Reason for Exclusion'])
    excluded_hitpath_requests = excluded_hitpath_requests.sort_values(by='Pieces in La Nina')

    return need_request, tf_30d_stats, excluded_hitpath_requests

In [17]:
def convert_number_to_letter(num_sheet_cols):
    """
    Takes the 1-indexed number of a column in a google sheet and returns the letter version. For example,
    if num_sheet_cols=1, it would return 'A'. If it was 27, it would return 'AA'. If it was 53, it would return 'BA'.

    :param num_sheet_cols: an int of the column in the google sheet
    :return: a string of the corresponding letter(s) of the column
    """
    # One letter cases (A-Z)
    if ((num_sheet_cols // 26) == 0) | (num_sheet_cols == 26):
        letter = chr(64 + num_sheet_cols)

    # Multiple letter cases (i.e. AA)
    else:
        # Z special case
        if num_sheet_cols % 26 == 0:
            first_letter = chr(64 + (num_sheet_cols // 26) - 1)
            second_letter = chr(64 + num_sheet_cols - (((num_sheet_cols // 26) - 1) * 26))
            letter = (first_letter + second_letter)
        # All other letters
        else:
            first_letter = chr(64 + (num_sheet_cols // 26))
            second_letter = chr(64 + num_sheet_cols - ((num_sheet_cols // 26) * 26))
            letter = (first_letter + second_letter)

    return letter

# Make DataFrames and Save as Excel

In [18]:
need_request, tf_30d_stats, excluded_hitpath_requests = make_final_dfs()

In [19]:
from nltk import flatten
from openpyxl.styles import numbers
from openpyxl import Workbook
from openpyxl.styles import Font, Color, colors, fills
from openpyxl.utils.dataframe import dataframe_to_rows

from openpyxl.styles.borders import Border, Side, BORDER_THIN
thin_border = Border(
    left=Side(border_style=BORDER_THIN, color='D3D3D3'),
    right=Side(border_style=BORDER_THIN, color='D3D3D3'),
    top=Side(border_style=BORDER_THIN, color='D3D3D3'),
    bottom=Side(border_style=BORDER_THIN, color='D3D3D3')
)

In [20]:
# Make Recently Tested Content Report
wb = Workbook()
ws = wb.active
all_sheets = []
ws_headers = {}

ws_headers['TF Content Requests'] = need_request.columns.tolist()
ws_headers['30 Day TF Stats'] = tf_30d_stats.columns.tolist()
ws_headers['Excluded Offers'] = excluded_hitpath_requests.columns.tolist()

content_request_sheet = wb['Sheet']
content_request_sheet.title = 'TF Content Requests'

# Column names to format specially
currency_format_cols = ['Revenue', 'Opportunity Cost', 'eCPM', 'Opportunity Cost eCPM']
percent_format_cols = ['OOR', 'CTR', 'Jump Page CTR', 'Top Creative Drop %']
comma_format_cols = ['Optout', 'Clicks', 'Jump Page Clicks', 'Delivered']

for r in dataframe_to_rows(need_request, index=False, header=True):
    content_request_sheet.append(r)
all_sheets.append(content_request_sheet)

tf_30d_stats_sheet = wb.create_sheet('30 Day TF Stats')
for r in dataframe_to_rows(tf_30d_stats, index=False, header=True):
    tf_30d_stats_sheet.append(r)
all_sheets.append(tf_30d_stats_sheet)

excluded_offers_sheet = wb.create_sheet('Excluded Offers')
for r in dataframe_to_rows(excluded_hitpath_requests, index=False, header=True):
    excluded_offers_sheet.append(r)
all_sheets.append(excluded_offers_sheet)

logic_and_definitions_sheet = wb.create_sheet('Logic and Definitions')
logic_and_definitions_sheet.cell(1, 1).value = 'Logic:'
logic_and_definitions_sheet.cell(1, 2).value = 'https://docs.google.com/document/d/1qULMVf6xEkUYNpf7t8FOCNIMSxoy_LQTL1L3VaPv-qk/edit?usp=sharing'
logic_and_definitions_sheet.cell(2, 1).value = 'Definitions:'
logic_and_definitions_sheet.cell(2, 2).value = 'https://docs.google.com/document/d/1al4IlJBpN3yN9_rD_TJuBnqHtWcjAI8ECBnuUw1Xiys/edit?usp=sharing'


for ws in all_sheets:
    dims = {}
    headers = ws_headers[ws.title]

    for row in ws.rows:
        for cell in ws["1:1"]:
            cell.font = Font(bold=True)
        for cell in row:
            newline_count = 1
            if cell.value:
                if type(cell.value)==str:
                    if ('\n' in cell.value):
                        newline_count = cell.value.count('\n')
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))/newline_count 
    for col, value in dims.items():
        ws.column_dimensions[col].width = value

    for col in currency_format_cols:
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE

    for col in percent_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_PERCENTAGE_00

    for col in comma_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = '#,##0'

    ws.freeze_panes = 'A2'   
save_path = filepath.output_folder + "Content Request Reports/"
today = date.today().strftime("%m_%d_%Y")
filename = os.path.join(save_path, f"TF Content Request Report - {today}.xlsx")
wb.save(filename)

In [21]:
import import_ipynb
from ipynb.fs.full.Content_Request_Report_SMS import *
request_tf = ['MFA', 'N3G', "FRH",'DA','TRS']
submission_df = finalize_submission_tf_df(need_request,request_tf )
final_submission_df = make_pieces_request_tf_df(submission_df,request_tf, request_limit=50)


/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/Content_Request_Report_SMS.ipynb:49: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  "metadata": {},
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

  Hitpath ID Request ID High Priority Submission Date  Due Date   \
0      13498                    False      2024-07-15 2024-07-18   
1      13349                    False      2024-07-15 2024-07-18   
2      11600                    False      2024-07-15 2024-07-18   
3      12710                    False      2024-07-15 2024-07-18   
4      13519                    False      2024-07-15 2024-07-18   

  Request Sent to Copywriter (Kellie)  \
0                                       
1                                       
2                                       
3                                       
4                                       

  Unapproved Content Document Link (Copywriter)  \
0                                                 
1                                                 
2                                                 
3                                                 
4                                                 

  Unapproved Jump Page Content (Copyw

In [28]:
#sheet_id = 8253683024220036 # real sheet id 
sheet_id = 7853583898726276 # test sheet id 
key = 'wf3Jr9OaAT5OE8gE8yctPqfmzrv6dUFATUODD'
#result = place_in_smartsheet(final_submission_df, key, sheet_id)

Adding Data to Smartsheet...
Data added to Smartsheet


In [27]:
final_submission_df

,Hitpath ID,Submission Date,Due Date,Request Type (Request),Creative Approval Status,Offer Type,Hitpath Offer ID,Offer Name,Vertical,Landing Page / Redirect Link,Compliance & Additional Notes (Request),# of Pieces Requested Per Shortcode,# of Pieces Requested Per Toll Free,Shortcode Abbreviations,Toll Free Abbreviations,Requested By,RX Rep,External AM Email,Channel,Platform
0,13498,2024-07-15,2024-07-18,Content,Yes - need advertiser approval,Existing - More Content,13498,McDonalds - US (SMS) 2039 - Point2Web,Sweepstakes,https://www.rentonion.com/rd/r.php?sid=13498&p...,Extra Content for direct traffic in Toll Free,0,2,,MFA\nN3G\nFRH\nDA\nTRS,Script,Haley Bush,kristine.o@point2web.com,TF,SS
1,13349,2024-07-15,2024-07-19,Content,Yes - need advertiser approval,Existing - More Content,13349,Trendn Daily - iphone 15,Sweepstakes,https://www.rentonion.com/rd/r.php?sid=13349&p...,Extra Content for direct traffic in Toll Free,0,2,,MFA\nN3G\nFRH\nDA\nTRS,Script,Rhiannon Selander,taylor@interest-media.com,TF,SS
2,11600,2024-07-15,2024-07-19,Content,Yes - need advertiser approval,Existing - More Content,11600,Unemployments Benefits Guide Stimulus Carousel...,Resources,https://www.rentonion.com/rd/r.php?sid=11600&p...,Extra Content for direct traffic in Toll Free,0,2,,MFA\nN3G\nFRH\nDA\nTRS,Script,Hartley Goode,Mike - mike@whatifmediagroup.com,TF,SS
3,12710,2024-07-15,2024-07-22,Content,Yes - need advertiser approval,Existing - More Content,12710,Unemployment Benefits Guide - State Stimulus -...,Resources,https://www.rentonion.com/rd/r.php?sid=12710&p...,Extra Content for direct traffic in Toll Free,0,2,,MFA\nN3G\nFRH\nDA\nTRS,Script,Hartley Goode,Mike - mike@whatifmediagroup.com,TF,SS
4,13519,2024-07-15,2024-07-22,Content,Yes - need advertiser approval,Existing - More Content,13519,Smart--Advances.com $1K - AvenueLink (SMS-OPS),Loan,https://www.rentonion.com/rd/r.php?sid=13519&p...,Extra Content for direct traffic in Toll Free,0,1,,MFA\nN3G\nFRH\nDA\nTRS,Script,Jennifer Sporleder,Ryan Zimmerman (ryan@avenuelink.com),TF,SS


# Send Email out

In [24]:
#email
""" 
toaddr = ['offernotices@rxmg.com', 'nathan@rxmg.com','nina@rxmg.com', 'k.smith@rxmg.com', 'a.miller@rxmg.com',  'lili@rxmg.com', 'r.woodward@rxmg.com', 'b.ratzlaff@rxmg.com']
subject_line = f"SMS Content Request Report - {today}"
email_body = "Hi Team,\n\n"
email_body += "Please find this week's Content Request Report attached.\n\n" 
email_body += "The following AMs have offer content that was requested:\n" 
for key, value in rx_rep_offer_dict.items():
    if value == 1:
        email_body += f"\t{key}: {value} offer\n"
    else:
        email_body += f"\t{key}: {value} offers\n"
email_body += f"\nThanks,\n{filepath.name}\n\n"

#send email
for i in toaddr:
    send_email.send_email([filename], subject_line, email_body, i)
"""

' \ntoaddr = [\'offernotices@rxmg.com\', \'nathan@rxmg.com\',\'nina@rxmg.com\', \'k.smith@rxmg.com\', \'a.miller@rxmg.com\',  \'lili@rxmg.com\', \'r.woodward@rxmg.com\', \'b.ratzlaff@rxmg.com\']\nsubject_line = f"SMS Content Request Report - {today}"\nemail_body = "Hi Team,\n\n"\nemail_body += "Please find this week\'s Content Request Report attached.\n\n" \nemail_body += "The following AMs have offer content that was requested:\n" \nfor key, value in rx_rep_offer_dict.items():\n    if value == 1:\n        email_body += f"\t{key}: {value} offer\n"\n    else:\n        email_body += f"\t{key}: {value} offers\n"\nemail_body += f"\nThanks,\n{filepath.name}\n\n"\n\n#send email\nfor i in toaddr:\n    send_email.send_email([filename], subject_line, email_body, i)\n'